In [7]:
import pandas as pd

In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('gender_submission.csv')

In [9]:
sample.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.363636
std,120.810458,0.481622
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


In [10]:
sample.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


**Cek sample submission.**
dari describe dan head dapat kita ketahui bahwa variabel terikat yang dihasilkan adalah 0 dan 1, artinya variabel terikat yaitu survived adalah data kategorik yang bersifat dikotomi (hanya berisi 2 kategori saja). Karena itu metode yang akan dipakai adalah regresi logistik karena regresi logistik mampu memprediksi data dikotomi. Sedangkan metode yang lain seperti random forest, decision tree, ANN kurang baik untuk data ini dikarenakan metode-metode tersebut digunakan untuk memproses data kategorik yang tidak dikotomi. Dan jika metode tersebut dipakai maka hasil yang diperoleh akan menjadi bias.

sifat-sifat regresi logistik dapat dilihat di web berikut : https://www.statistikian.com/2015/02/regresi-logistik.html

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [13]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
# drop cabin karena terlalu banyak data yang hilang
train = train.drop(['Cabin'], axis=1)
test = test.drop(['Cabin'], axis=1)

In [16]:
# fill missing value dengan median untuk data nominal
# fill missing value dengan modos untuk data kategorik
train['Age'] = train['Age'].fillna(train['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [17]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


**here we got some problem.**
variabel 'name' dan variabel 'ticket' sulit untuk diterjemahkan ke dalam bahasa pemrograman sehingga kita harus berasumsi apakah variabel lain dapat mewakilkan variabel 'name' atau 'ticket' ?. variabel nama dapat diwakilkan dengan sex dan age, sedangkan variabel ticket dapat diwakilkan dengan fare dan embarked. Sehingga kita perlu menghapus variabel 'name' dan 'ticket' karena kita sudah tidak membutuhkan data tersebut.

In [18]:
train = train.drop(['Name', 'Ticket'], axis=1)
test = test.drop(['Name', 'Ticket'], axis=1)

In [19]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder() 
#train['Cat_Sex']= le.fit_transform(train['Sex']) 
#train['Cat_Embarked']= le.fit_transform(train['Embarked']) 
#test['Cat_Sex']= le.fit_transform(test['Sex']) 
#test['Cat_Embarked']= le.fit_transform(test['Embarked']) 

In [20]:
# mengganti data kategorik ke dalam numerik dengan kolom yang berbeda-beda
male = []
female = []
for num in train['Sex']:
    if num == 'male':
        male.append(1)
        female.append(0)
    else:
        male.append(0)
        female.append(1)
train['male'] = male
train['female'] = female

male = []
female = []
for num in test['Sex']:
    if num == 'male':
        male.append(1)
        female.append(0)
    else:
        male.append(0)
        female.append(1)
test['male'] = male
test['female'] = female

S = []
C = []
Q = []
for num in train['Embarked']:
    if num == 'S':
        S.append(1)
        C.append(0)
        Q.append(0)
    elif num == 'C':
        S.append(0)
        C.append(1)
        Q.append(0)
    else:
        S.append(0)
        C.append(0)
        Q.append(1)
train['S'] = S
train['C'] = C
train['Q'] = Q

S = []
C = []
Q = []
for num in test['Embarked']:
    if num == 'S':
        S.append(1)
        C.append(0)
        Q.append(0)
    elif num == 'C':
        S.append(0)
        C.append(1)
        Q.append(0)
    else:
        S.append(0)
        C.append(0)
        Q.append(1)
test['S'] = S
test['C'] = C
test['Q'] = Q

In [21]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,male,female,S,C,Q
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,1,0,0
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,1,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,1,1,0,0
3,4,1,1,female,35.0,1,0,53.1000,S,0,1,1,0,0
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,1,0,0


In [22]:
# buang variabel sex dan embarked karena kita sudah tidak memerlukan lagi
train = train.drop(['Sex', 'Embarked'], axis=1)
test = test.drop(['Sex', 'Embarked'], axis=1)

In [23]:
# simpan variabel PassengerId karena nanti akan digunakan di submission sebagai id-nya
PassengerId_test = test['PassengerId']

In [24]:
# karena kita sudah menyimpan data id maka kita akan buang PassengerId karena id tidak berpengaruh apa-apa 
# karena PassengerId hanya sebagai identitas saja
train = train.drop(['PassengerId'], axis=1)
test = test.drop(['PassengerId'], axis=1)

In [25]:
# sebelum mencoba dengan data test, model perlu di coba terlebih dahulu kepada sample
# maka untuk mencari sample nya digunakan train test split
X = train.drop(['Survived'], axis=1)
Y = train['Survived']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [32]:
# mencoba untuk melihat akurasi dengan data sample
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs')
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)

C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [33]:
# melihat hasil 
# confusion_matrix, classification_report berguna untuk melihat akurasi data hasil prediksi
# accuracy_score untuk melihat accurasi
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,accuracy_score
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))
print(accuracy_score(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.82      0.87      0.84       157
           1       0.79      0.73      0.76       111

   micro avg       0.81      0.81      0.81       268
   macro avg       0.81      0.80      0.80       268
weighted avg       0.81      0.81      0.81       268

[[136  21]
 [ 30  81]]
0.8097014925373134


In [34]:
# parameter tuning berguna untuk memaksimalkan akurasi
# pemilihan C berdasarkan akurasi test yang tidak berbeda jauh dengan akurasi train
# hal ini digunakan agar model dapat memprediksi data yang tidak terlihat
# tolak ukur dalam pemilihan model adalah akurasi score +- 10 antara akurasi train dan akurasi test
C_param = [0.001,0.01,0.1,1,10,100,1000]
for num in C_param:
    model=LogisticRegression(C=num, solver='lbfgs')
    
    model.fit(X_train,Y_train)
    Y_pred = model.predict(X_test)
    a = accuracy_score(Y_test, Y_pred)
    
    model.fit(X_test,Y_test)
    Y_pred = model.predict(X_train)
    b = accuracy_score(Y_train, Y_pred)
    
    selisih = abs(b-a)
    print(a,b, selisih, num)

0.6604477611940298 0.680577849117175 0.02013008792314519 0.001
0.753731343283582 0.7271268057784912 0.02660453750509084 0.01


C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8059701492537313 0.7849117174959872 0.021058431757744134 0.1
0.8097014925373134 0.78330658105939 0.02639491147792339 1


C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8134328358208955 0.7865168539325843 0.026915981888311236 10
0.8134328358208955 0.78330658105939 0.030126254761505544 100
0.8134328358208955 0.78330658105939 0.030126254761505544 1000


C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [35]:
# saya memilih 0.1 karena saya membutuhkan akurasi dan selisih akurasi score train dan test tidak berbeda jauh
# maka kita lakukan pemodelan kedalam data aslinya
model=LogisticRegression(C=0.1, solver='lbfgs')
model.fit(X,Y)
Y_pred = model.predict(test)

C:\Users\daniel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [36]:
# memasukkan hasil prediksi ke dalam data frame sesuai dengan nama kolom sample submissin yang diberikan di awal
submission = pd.DataFrame({
    'PassengerId' : PassengerId_test,
    'Survived' : Y_pred
})

In [37]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


**Apabila ada yang mau ditanyakan atau mengkritik atau memberi saran. silahkan email : daniel.pnjt@gmail.com.
Thankyou.**